# SQL Lab: 03 ctes and funnels

In [ ]:
import duckdb, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')


In [ ]:
# Initialize DuckDB in-memory database and load schema + seed data
conn = duckdb.connect(database=':memory:')
conn.execute(open('sql/schema.sql').read())
conn.execute(open('sql/seed.sql').read())
print('Tables loaded:', conn.execute('SHOW TABLES').fetchall())


In [ ]:
    # Peek at each table to validate load
    tables = ['customers','products','orders','order_items','events','marketing_experiments']
    for t in tables:
        print(f"
Preview of {t}:")
        display(conn.execute(f'SELECT * FROM {t} LIMIT 5').df())


In [ ]:
# Build sessionized funnel using CTEs
query = '''
    WITH visits AS (
        SELECT customer_id, min(event_ts) AS visit_ts
        FROM events
        WHERE event_type = 'visit'
        GROUP BY 1
    ), signups AS (
        SELECT customer_id, min(event_ts) AS signup_ts
        FROM events
        WHERE event_type = 'signup'
        GROUP BY 1
    ), purchases AS (
        SELECT customer_id, min(order_ts) AS purchase_ts
        FROM orders
        GROUP BY 1
    ), funnel AS (
        SELECT v.customer_id, v.visit_ts, s.signup_ts, p.purchase_ts
        FROM visits v
        LEFT JOIN signups s USING (customer_id)
        LEFT JOIN purchases p USING (customer_id)
    )
    SELECT * FROM funnel
'''
funnel = conn.execute(query).df()
funnel.head()


In [ ]:
# Compute step-through rates
step_counts = {
    'visit': len(funnel),
    'signup': funnel['signup_ts'].notna().sum(),
    'purchase': funnel['purchase_ts'].notna().sum(),
}
step_counts


In [ ]:
# Funnel chart
fig, ax = plt.subplots(figsize=(6,4))
steps = list(step_counts.keys())
values = list(step_counts.values())
ax.bar(steps, values, color=['#60BD68', '#5DA5DA', '#FAA43A'])
ax.set_title('Visit → Signup → Purchase Funnel')
ax.set_ylabel('Users')
plt.tight_layout()


In [ ]:
# Sessionized conversion times using CTEs
query = '''
    WITH ordered_events AS (
        SELECT customer_id, event_type, event_ts,
               LAG(event_ts) OVER (PARTITION BY customer_id ORDER BY event_ts) AS prev_ts
        FROM events
    )
    SELECT event_type,
           AVG(EXTRACT('epoch' FROM (event_ts - prev_ts)))/3600 AS avg_hours_since_prior
    FROM ordered_events
    WHERE prev_ts IS NOT NULL
    GROUP BY 1
'''
lag_stats = conn.execute(query).df()
lag_stats


In [ ]:
# Visualize average hours between events
fig, ax = plt.subplots(figsize=(6,4))
sns.barplot(data=lag_stats, x='event_type', y='avg_hours_since_prior', ax=ax, palette='viridis')
ax.set_title('Average Hours Between Events')
plt.tight_layout()
